In [288]:
import pandas as pd
import itertools
import numpy as np
import openpyxl
# from scenario_generation import generate_fuel_prices


### SETS

In [289]:
fuel_price_csv = 'fuel_price2_4s.csv'
# Velg antall av de ulike settene her
#set_sheets = {'Power systems': 11, 'Ages': 6, 'Routes': 3122, 'Time periods':6, 'Time periods 1': 2, 'Time periods 2':4, 'Fuel types': 10, 'Installations': 18, 'Scenarios': 100}

t_values = range(0, 6)  # time period from 1 to 6
t1_values = range(0, 2)
t2_values = range(2, 6)
tp2_values = range(2,7)
f_values = range(1, 10)  # Assuming you want values from 1 to 7 for 'f'
r_values = range(1, 1819)  # Assuming you want values from 1 to 3122 for 'r'
n_values = range(1, 19) # 18 installations
s_values = range(1,12) # 11 systems
w_values = range(1, 5) # 100 scenarios
a_values = range(0, 6)

for t in f_values:
    print(t)


1
2
3
4
5
6
7
8
9


In [290]:
# Skriver alle sets med verdier definert over til excel

sets_excel_output_path = '../run_model/SetData-sheets.xlsx'

set_sheets = {
    'Power systems': s_values,
    'Ages': a_values,
    'Routes': r_values,
    'Time periods': t_values,
    'Time periods 1': t1_values,
    'Time periods 2': t2_values,
    'TP2 scrap': tp2_values,
    'Fuel types': f_values,
    'Installations': n_values,
    'Scenarios': w_values
}

# Create data frames for each set and write to Excel
with pd.ExcelWriter(sets_excel_output_path, engine='openpyxl') as writer:
    for sheet, values in set_sheets.items():
        # Create a DataFrame for the current set
        df = pd.DataFrame(itertools.product(values), columns=[sheet])

        # Write the DataFrame to the Excel file with the current sheet name
        df.to_excel(writer, sheet_name=sheet, index=False)

#### Generere kombinasjoner av indekser til parameterne

In [291]:

combinations_f_r = itertools.product(f_values, r_values)
combinations_ems = itertools.product(f_values, r_values)
combinations_r_s = itertools.product(r_values, s_values)
combinations_t = itertools.product(t_values)
combinations_s_a = itertools.product(s_values, a_values)
combinations_f_r_w = itertools.product(f_values, r_values, w_values)
combinations_w = itertools.product(w_values)
combinations_r = itertools.product(r_values)

#### Inputdata

In [292]:
# Hardkodet inn fuelpris per MJ per fueltype 1-6
fuel_prices = {1: 0.171, 2:0.163, 3:0.514, 4:0.328, 5:0.222, 6:0.205, 7: 0.197, 8: 0.183, 9: 0.297}




# fuel_dict = {
#     1: (0.178, 0.094, 'fossil'),      # MGO
#     2: (0.146, 0.073, 'fossil'),      # LNG
#     3: (0.491, 0.261, 'green_fuels'), # GREEN METHANOL
#     4: (0.324, 0.178, 'green_fuels'), # GREEN NH3
#     5: (0.278, 0.147, 'bio'),         # B30
#     6: (0.245, 0.130, 'bio'),         # B20
#     7: (0.228, 0.121, 'bio'),         # B15
#     8: (0.201, 0.106, 'bio'),         # B7
#     9: (0.301, 0.129, 'green_fuels')  # Helelektrisk
# }
 # Example fuel bounds with group information


# Karbonutslipp per MJ som er brukt for hver type fuel
emissions_carbon_tonn_per_MJ_wtw = {1: 0.000182, 2: 0.000153, 3: 0.000033, 4: 0.000033, 5: 0.000134, 6: 0.000150, 7: 0.000158, 8: 0.000170, 9: 0.000008 }
emissions_carbon_tonn_per_MJ_ttw = {1: 0.000152778, 2: 0.000115556, 3: 0.000085278, 4: 0.000025583, 5: 0.000017528, 6: 0.0, 7: 0.0}
emissions_carbon_tonn_per_MJ = emissions_carbon_tonn_per_MJ_wtw 


# Regne ut reduksjonskrav for karbonutslipp
init_ems_tonn = 165000

scenario_1_reductions = {0: 0, 1: 0.50, 2: 0.55, 3: 0.70, 4: 0.80, 5: 0.80 } # ambisjonene til norske myndigheter. t=1 = 2030
scenario_2_reductions = {0: 0, 1: 0, 2: 0.29, 3: 0.49, 4: 0.65, 5: 0.78, 6: 0.87} # DNV report Norway 2023 - kutt kun innenfor transport
scenario_0_reductions = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0}

# Velg hvilket scenario som skal kjøres
reduction_requirements = scenario_1_reductions

capacity_systems = {1: 100, 2: 100, 3: 100, 4: 100, 5: 70, 6: 70, 7: 70, 8: 70, 9: 90, 10: 90, 11: 70, 12: 90, 13: 90}

#### Lese inn data fra rutegenereringsprogrammet

In [293]:
# # Leser inn nødvendig MJ på å kjøre hver rute
mj_route = pd.read_csv('../route_generation/generated_datafiles/mj_route.csv')
mj_route_el = pd.read_csv('../route_generation/generated_datafiles/mj_route_strøm.csv')



# # Leser inn tid på å kjøre hver rute
hours_route = pd.read_csv('../route_generation/generated_datafiles/hours_route.csv')
hours_route_el = pd.read_csv('../route_generation/generated_datafiles/hours_route_strøm.csv')

demand_el = pd.read_csv('../route_generation/generated_datafiles/demand_strøm.csv')
demand = pd.read_csv('../route_generation/generated_datafiles/demand.csv')

# Skriver verdiene tilbake i excelfilen for alle parametrene
output_excel_path = '../run_model/Parameterdata-sheets.xlsx'

##### Generere P_w

In [294]:
df_probability = pd.DataFrame(combinations_w, columns=['O'])

df_probability['Value'] = 1/len(list(w_values))

df_probability.head()


,O,Value
0,1,0.2
1,2,0.2
2,3,0.2
3,4,0.2
4,5,0.2


In [295]:

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_probability.to_excel(writer, sheet_name='Probability', index=False)

##### Distance dict for compatibility og cost of fuel

In [296]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()

# lager egne distanser for elektrisk

# specific_distance_dict = mj_route_el['mj_route'].to_dict()
#specific_distance_dict= specific_distance_dict['mj_route']
# electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(len(distance_dict))
mj_route



1818


,mj_route
0,221734.4
1,194299.6
2,199485.2
3,178969.6
4,202378.0
...,...
1813,261387.2
1814,254428.8
1815,260908.8
1816,244592.4


##### Compatibility RS

In [297]:
# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
demand_dict = demand['Demand'].to_dict()
#demand_dict = demand_dict['Demand']

# lager egne distanser for elektrisk

#specific_demand_dict = demand_el['Demand'].to_dict()
#specific_demand_dict= specific_demand_dict['Demand']

#electric_demand_dict = {key + 1: value for key, value in specific_demand_dict.items()}

demand_dict = {key + 1: value for key, value in demand_dict.items()}
print(demand_dict)
demand



{1: 85.25, 2: 92.25, 3: 71.75, 4: 76.0, 5: 86.75, 6: 96.25, 7: 75.75, 8: 96.25, 9: 75.75, 10: 81.25, 11: 81.25, 12: 95.92, 13: 85.25, 14: 95.75, 15: 85.25, 16: 96.25, 17: 90.75, 18: 94.25, 19: 78.25, 20: 75.42, 21: 82.75, 22: 75.25, 23: 75.75, 24: 70.25, 25: 73.75, 26: 83.0, 27: 72.0, 28: 72.0, 29: 71.67, 30: 79.0, 31: 71.5, 32: 89.75, 33: 72.0, 34: 87.0, 35: 87.0, 36: 72.0, 37: 72.0, 38: 86.67, 39: 76.0, 40: 94.0, 41: 86.5, 42: 76.0, 43: 87.0, 44: 81.5, 45: 85.0, 46: 76.0, 47: 75.67, 48: 83.0, 49: 75.5, 50: 93.75, 51: 76.0, 52: 70.5, 53: 74.0, 54: 75.67, 55: 83.0, 56: 75.5, 57: 93.75, 58: 76.0, 59: 70.5, 60: 74.0, 61: 99.25, 62: 99.25, 63: 96.25, 64: 78.75, 65: 78.75, 66: 82.67, 67: 75.17, 68: 93.42, 69: 75.67, 70: 70.17, 71: 73.67, 72: 72.0, 73: 82.75, 74: 82.5, 75: 72.0, 76: 83.0, 77: 77.5, 78: 81.0, 79: 93.25, 80: 75.5, 81: 73.5, 82: 82.75, 83: 93.75, 84: 88.25, 85: 91.75, 86: 75.75, 87: 70.5, 88: 74.0, 89: 94.0, 90: 94.0, 91: 79.0, 92: 79.0, 93: 93.67, 94: 83.0, 95: 93.5, 96: 83.0

,Demand
0,85.25
1,92.25
2,71.75
3,76.00
4,86.75
...,...
1813,93.00
1814,98.50
1815,98.50
1816,100.00


In [298]:
# Lage combinations for rs
# For hver rad med r s, sjekke om kapasiteten i s er større enn eller lik demanden til r. Sett til 1 hvis true, 0 ellers

# Create a DataFrame from the combinations
df_routecap = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Display the first few rows of the DataFrame
print(df_routecap.head())

df_routecap['Value'] = 0

# Iterate over the DataFrame
for index, row in df_routecap.iterrows():
    r = row['R']
    s = row['S']
    if demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
        df_routecap.at[index, 'Value'] = 1
    # if s == 11 and electric_demand_dict.get(r, 0) <= capacity_systems.get(s, float('inf')):
    #     df_routecap.at[index, 'Value'] = 1

#print(df_routecap)

   R  S
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [299]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_routecap.to_excel(writer, sheet_name='Compatibility rs', index=False)

##### Cost of fuel per route

In [300]:

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
distance_dict = mj_route['mj_route'].to_dict()
#distance_dict = distance_dict['mj_route']

# lager egne distanser for elektrisk

# specific_distance_dict = mj_route_el['mj_route'].to_dict()
# #specific_distance_dict= specific_distance_dict['mj_route']
# electric_distance_dict = {key + 1: value for key, value in specific_distance_dict.items()}

#print(distance_dict)
distance_dict = {key + 1: value for key, value in distance_dict.items()}
print(len(distance_dict))

print(distance_dict)
#print(len(electric_distance_dict))

1818
{1: 221734.4, 2: 194299.6, 3: 199485.2, 4: 178969.6, 5: 202378.0, 6: 206869.2, 7: 208123.2, 8: 210389.2, 9: 214603.2, 10: 181976.4, 11: 220056.4, 12: 188651.2, 13: 199334.4, 14: 199588.0, 15: 213814.4, 16: 200709.2, 17: 191107.6, 18: 186464.4, 19: 190129.2, 20: 200076.8, 21: 219008.4, 22: 211242.0, 23: 199803.2, 24: 203961.6, 25: 199638.4, 26: 222494.8, 27: 237840.0, 28: 243280.0, 29: 215862.0, 30: 233205.2, 31: 226078.8, 32: 235025.2, 33: 229680.0, 34: 264332.8, 35: 260332.8, 36: 232400.0, 37: 271920.0, 38: 243326.4, 39: 239769.6, 40: 248589.6, 41: 236811.6, 42: 254249.6, 43: 262572.8, 44: 216331.2, 45: 207699.6, 46: 244649.6, 47: 246031.6, 48: 266254.8, 49: 256968.4, 50: 253983.2, 51: 242809.6, 52: 247596.4, 53: 242084.8, 54: 255791.6, 55: 275694.8, 56: 265288.4, 57: 266943.2, 58: 255689.6, 59: 253916.4, 60: 247444.8, 61: 224876.4, 62: 262876.4, 63: 233029.2, 64: 238690.4, 65: 238530.4, 66: 225476.8, 67: 216510.4, 68: 239536.8, 69: 232751.6, 70: 209230.0, 71: 205546.8, 72: 17768

In [301]:
# Convert dictionaries to DataFrames
df_distance = pd.DataFrame(distance_dict.items(), columns=['R', 'Value'])
#df_specific_distance = pd.DataFrame(electric_distance_dict.items(), columns=['R', 'Value'])

# Sort DataFrames by 'R' values
df_distance = df_distance.sort_values(by='R')
#df_specific_distance = df_specific_distance.sort_values(by='R')


# print("Distance Dictionary DataFrame:")
# print(df_distance)

# print("\nSpecific Distance Dictionary DataFrame:")
# print(df_specific_distance)

with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_distance.to_excel(writer, sheet_name='Distance', index=False)

# with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
#     df_specific_distance.to_excel(writer, sheet_name='Specific distance', index=False)

Generere alle kombinasjoner av R og F

In [302]:
# Create a DataFrame from the combinations
df_fuelprice = pd.DataFrame(combinations_f_r, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_fuelprice.head())




   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


#### Fuelcost

In [303]:

# regner ut verdi for hver kombinasjon av fueltype og rute gitt verdiene i dictionariene over
# def calculate_value(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return fuel_prices[row['F']] * distance

def calculate_value(row):
    distance = distance_dict[row['R']]
    return fuel_prices[row['F']] * distance

# Legger til verdiene i Value-kolonne i dataframen
df_fuelprice['Value'] = df_fuelprice.apply(calculate_value, axis=1)

# df_fuelprice

##### Fuelcost 2

In [304]:
fuel_price2_df = pd.read_csv(fuel_price_csv)

fuel_price2_df.head()

,F,T,S,Value
0,1,2,1,0.09000
1,1,3,1,0.08766
2,1,4,1,0.08532
3,1,5,1,0.08298
4,1,2,2,0.09000


##### Skriver Cost of fuel til excel

In [305]:
# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    fuel_price2_df.to_excel(writer, sheet_name='Cost of fuel 2', index=False)

# Using ExcelWriter to write to the existing Excel file
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_fuelprice.to_excel(writer, sheet_name='Cost of fuel 1', index=False)

##### EMISSIONS FOR FUEL TYPE

Generere alle kombinasjoner av F og R i kolonnene


In [306]:
# Create a DataFrame from the combinations
df_ems = pd.DataFrame(combinations_ems, columns=['F', 'R'])

# Display the first few rows of the DataFrame
print(df_ems.head())

   F  R
0  1  1
1  1  2
2  1  3
3  1  4
4  1  5


In [307]:
# multipliserer carbon per MJ per fueltype med distanse seilt per rute. 
# def calculate_emissions(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return emissions_carbon_tonn_per_MJ[row['F']] * distance

# def calculate_emissions(row):
#     distance = electric_distance_dict[row['R']] if row['F'] == 6 else distance_dict[row['R']]
#     return emissions_carbon_tonn_per_MJ[row['F']] * distance
def calculate_emissions(row):
    distance =  distance_dict[row['R']]
    return emissions_carbon_tonn_per_MJ[row['F']] * distance



# Apply the function to create a new column 'Value'
df_ems['Value'] = df_ems.apply(calculate_emissions, axis=1)


# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_ems.to_excel(writer, sheet_name='Emissions', index=False)


##### MAX EMISSIONS PER TIME PERIOD

In [308]:


# Create a DataFrame from the combinations
df_max_ems = pd.DataFrame(combinations_t, columns=['T'])

# Display the first few rows of the DataFrame
# print(df_max_ems)

In [309]:


# Regner ut max emissions gitt uslippsreduksjonsscenario. Endre scenario-funksjon etter hvilket scenario vi vil kjøre med
def calculate_max_emissions(row):
    return (1 - reduction_requirements[row['T']]) * init_ems_tonn

# Legger til verdiene i Value-kolonne i dataframen
df_max_ems['Value'] = df_max_ems.apply(calculate_max_emissions, axis=1)
# print(df_max_ems.head())
# Skriver dataframen med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_max_ems.to_excel(writer, sheet_name='Max Emissions', index=False)

df_max_ems

,T,Value
0,0,165000.0
1,1,82500.0
2,2,74250.0
3,3,49500.0
4,4,33000.0
5,5,33000.0


##### TIME USED

In [310]:
# Create T_rs
combinations_r_s = itertools.product(r_values, s_values)
df_timeused = pd.DataFrame(combinations_r_s, columns=['R', 'S'])

# Lager dictionaries for MJ_route for å senere kunne multiplisere sammen disse verdiene for å få kostnad per rute per fueltype
route_dict = hours_route['Hours'].to_dict()
hours_route_dict = {key + 1: value for key, value in route_dict.items()}
print(hours_route_dict)
# lager egne distanser for elektrisk
# hours_route_el_dict = hours_route_el['Hours'].to_dict()
# hours_route_el_dict = {key + 1: value for key, value in hours_route_el_dict.items()}
# print(hours_route_el_dict)


def calculate_timeused(row):
    value =  hours_route_dict[row['R']]
    return value


df_timeused['Value'] = df_timeused.apply(calculate_timeused, axis=1)

print(df_timeused.head())
# Skriver dataframe med emissions til parameter_data excel filen
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_timeused.to_excel(writer, sheet_name='Time used', index=False)


{1: 48.0, 2: 48.0, 3: 48.0, 4: 48.0, 5: 48.0, 6: 48.0, 7: 48.0, 8: 48.0, 9: 48.0, 10: 48.0, 11: 48.0, 12: 48.0, 13: 48.0, 14: 48.0, 15: 48.0, 16: 48.0, 17: 48.0, 18: 48.0, 19: 48.0, 20: 48.0, 21: 48.0, 22: 48.0, 23: 48.0, 24: 48.0, 25: 48.0, 26: 48.0, 27: 48.0, 28: 48.0, 29: 48.0, 30: 48.0, 31: 48.0, 32: 48.0, 33: 48.0, 34: 48.0, 35: 48.0, 36: 48.0, 37: 48.0, 38: 48.0, 39: 48.0, 40: 48.0, 41: 48.0, 42: 48.0, 43: 48.0, 44: 48.0, 45: 48.0, 46: 48.0, 47: 48.0, 48: 48.0, 49: 48.0, 50: 48.0, 51: 48.0, 52: 48.0, 53: 48.0, 54: 48.0, 55: 48.0, 56: 48.0, 57: 48.0, 58: 48.0, 59: 48.0, 60: 48.0, 61: 48.0, 62: 48.0, 63: 48.0, 64: 48.0, 65: 48.0, 66: 48.0, 67: 48.0, 68: 48.0, 69: 48.0, 70: 48.0, 71: 48.0, 72: 48.0, 73: 48.0, 74: 48.0, 75: 48.0, 76: 48.0, 77: 48.0, 78: 48.0, 79: 48.0, 80: 48.0, 81: 48.0, 82: 48.0, 83: 48.0, 84: 48.0, 85: 48.0, 86: 48.0, 87: 48.0, 88: 48.0, 89: 48.0, 90: 48.0, 91: 48.0, 92: 48.0, 93: 48.0, 94: 48.0, 95: 48.0, 96: 48.0, 97: 48.0, 98: 48.0, 99: 48.0, 100: 48.0, 101: 48

#### Installations in route

In [311]:
import csv

csv_file = "../route_generation/generated_datafiles/routes.csv"
excel_file = '../run_model/Parameterdata-sheets.xlsx'

# Open the existing Excel workbook
workbook = openpyxl.load_workbook(excel_file)
sheet = workbook["Installations in route"]

# Open the CSV file in text mode and read all lines
with open(csv_file, 'r') as file:
    lines = file.readlines()

# Add column headers if necessary
if not sheet['A1'].value:
    sheet['A1'] = "i"
    sheet['B1'] = "r"
    sheet['C1'] = "Value"

# Initialize the row index for Excel
row_index = 2

# Initialize an empty dictionary to store the platform mapping
platform_mapping = {}

# Path to your CSV file
csv_file = "../route_generation/clustering/output_platforms_demand.csv"

# Open the CSV file and read the contents
with open(csv_file, newline='') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Skip the header row
    for row in reader:
        platform = row[0]  # Platform name is in the first column
        # Assign a number to each platform
        platform_mapping[platform] = len(platform_mapping) + 1

print(platform_mapping)


{'APT': 1, 'ASL': 2, 'DAB': 3, 'DSA': 4, 'DSS': 5, 'GFAGFBGFC': 6, 'KVB': 7, 'MID': 8, 'NLNVAL': 9, 'OSC': 10, 'OSE': 11, 'OSOVFB': 12, 'OSS': 13, 'STASTB': 14, 'STC': 15, 'TENTRB': 16, 'TEQTRC': 17, 'TRO': 18}


In [312]:

platform_mapping_reverse = {v: k for k, v in platform_mapping.items()}

length = len(platform_mapping) +1

# Process each line in the CSV file
for idx, line in enumerate(lines, start=1):
    for i in range(1, length):
        sheet.cell(row=row_index, column=2, value=idx)
        sheet.cell(row=row_index, column=1, value=i)
        platform_name = platform_mapping_reverse.get(i, "")  # Get the platform name
        if platform_name in line:
            sheet.cell(row=row_index, column=3, value=1)
        else:
            sheet.cell(row=row_index, column=3, value=0)
        row_index += 1

workbook.save(excel_file)
print("Data has been successfully written.")

Data has been successfully written.


In [313]:

csv_file = '../route_generation/clustering/output_platforms_visits.csv'


visits_per_week = {}
with open(csv_file, newline='') as file:
    reader = csv.reader(file, delimiter=';')
    next(reader)  # Skip the header row
    i = 1
    for row in reader:
        platform = row[0]  # Platform name is in the first column
        # Assign a number to each platform
        # Assign a number to each platform
        visits_per_week[i] = row[1]
        i += 1

i_values = range(1, len(platform_mapping)+1)
print(visits_per_week)

combinations_i_t = itertools.product(i_values, t_values)
df_demand = pd.DataFrame(combinations_i_t, columns=['I', 'T'])
# Iterate over rows in the DataFrame
df_demand['Value'] = df_demand['I'].map(lambda x: visits_per_week[x])



df_demand

{1: '2', 2: '3', 3: '4', 4: '3', 5: '3', 6: '4', 7: '2', 8: '2', 9: '3', 10: '2', 11: '5', 12: '4', 13: '3', 14: '4', 15: '3', 16: '4', 17: '4', 18: '2'}


,I,T,Value
0,1,0,2
1,1,1,2
2,1,2,2
3,1,3,2
4,1,4,2
...,...,...,...
103,18,1,2
104,18,2,2
105,18,3,2
106,18,4,2


In [314]:
with pd.ExcelWriter(output_excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_demand.to_excel(writer, sheet_name='Demand', index=False)